In [1131]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import torch 
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [2]:
aws_df_raw = pd.read_csv("...csv path...")

In [3]:
aws_df_rec = pd.read_csv("...csv path...")

In [1006]:
aws_df_rec2 = pd.read_csv("...csv path...")

In [1278]:
aws_df_rec3 = pd.read_csv("...csv path...")

In [1279]:
aws_df_raw['time'] = pd.to_datetime(aws_df_raw['time'], utc=True)
aws_df_rec['time'] = pd.to_datetime(aws_df_rec['time'], utc=True)
aws_df_rec2['time'] = pd.to_datetime(aws_df_rec2['time'], utc=True)
aws_df_rec3['time'] = pd.to_datetime(aws_df_rec3['time'], utc=True)

In [1280]:
start_date ='2022-08-25'
aws_df_raw = aws_df_raw[aws_df_raw['time'] >= start_date]s

# 결과 확인
aws_df_raw

,SpotPrice,SPS,AZ,Region,InstanceType,IF,time
7612824,1.7565,3,apne1-az4,ap-northeast-1,r5dn.metal,0.0,2022-08-25 00:00:00+00:00
7612825,0.3383,3,apne1-az4,ap-northeast-1,r6i.4xlarge,1.0,2022-08-25 00:00:00+00:00
7612826,1.3203,3,cac1-az4,ca-central-1,d3.8xlarge,1.5,2022-08-25 00:00:00+00:00
7612827,1.6226,2,usw2-az4,us-west-2,c6i.24xlarge,2.0,2022-08-25 00:00:00+00:00
7612828,1.7565,2,apne1-az2,ap-northeast-1,r5dn.metal,0.0,2022-08-25 00:00:00+00:00
...,...,...,...,...,...,...,...
29706394,1.1561,3,euc1-az3,eu-central-1,m5d.12xlarge,2.5,2023-05-10 23:50:00+00:00
29706395,0.4602,3,euw1-az1,eu-west-1,a1.metal,2.5,2023-05-10 23:50:00+00:00
29706396,1.0457,3,usw1-az1,us-west-1,r6g.12xlarge,1.0,2023-05-10 23:50:00+00:00
29706397,1.7306,3,usw2-az1,us-west-2,c5d.12xlarge,3.0,2023-05-10 23:50:00+00:00


In [1015]:
aws_df = pd.concat([aws_df_raw, aws_df_rec])

In [1281]:
aws_df = pd.concat([aws_df, aws_df_rec2, aws_df_rec3])

In [1282]:
from collections import Counter

group_A_dfs = []
instance_list = ['c4.2xlarge', 'c4.4xlarge', 'c4.large', 'c4.xlarge', 'd2.2xlarge', 'd2.4xlarge', 'd2.xlarge', 'm4.2xlarge', 'm4.4xlarge', 'm4.large', 'm4.xlarge', 't2.2xlarge', 't2.xlarge']
az_list = ['apne2-az1', 'apne2-az2', 'apne2-az3']

for instance in instance_list:
    for az in az_list:
        temp_df = aws_df[(aws_df['InstanceType']==instance) & (aws_df['Region']=='ap-northeast-2') & (aws_df['AZ']==az)]
        if len(temp_df) > 0:
            group_A_dfs.append(temp_df)

In [1283]:
SPS_scaler = MinMaxScaler(feature_range=(1, 10))
s_scaler = SPS_scaler.fit(group_A_dfs[1][['SPS']])

IF_scaler = MinMaxScaler(feature_range=(1, 10))
i_scaler = IF_scaler.fit(group_A_dfs[3][['IF']])

In [1284]:
df1 = group_A_dfs[20]

In [1285]:
df1 = df1.groupby(["InstanceType"])['time'].apply(lambda x:pd. date_range(start=x.min() , end=x.max(), freq="10min")).explode ().reset_index().merge(df1, how='left').ffill()

In [1286]:
df1['SpotPrice'] = df1['SpotPrice'].replace(-1, method='ffill')
df1['SpotPrice'] = df1['SpotPrice'].replace(0, method='ffill')
df1['SPS'] = df1['SPS'].replace(-1, method='ffill')
df1['SPS'] = df1['SPS'].replace(0, method='ffill')
df1['IF'] = df1['IF'].replace(-1, method='ffill')
df1['IF'] = df1['IF'].replace(0, method='ffill')

In [1287]:
df1[:5]

,InstanceType,time,SpotPrice,SPS,AZ,Region,IF
0,m4.4xlarge,2022-08-25 00:00:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0
1,m4.4xlarge,2022-08-25 00:10:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0
2,m4.4xlarge,2022-08-25 00:20:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0
3,m4.4xlarge,2022-08-25 00:30:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0
4,m4.4xlarge,2022-08-25 00:40:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0


### create feature

In [1288]:
import datetime

df1['weekday'] = df1['time'].apply(lambda x: datetime.datetime.strftime(x, '%A'))

In [1289]:
# One-hot Encoding 
weekday_onehot = pd.get_dummies(df1['weekday'], prefix='weekday')
df1 = pd.concat([df1, weekday_onehot], axis=1)

In [1290]:
weekday_columns = ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday',
                   'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']

for column in weekday_columns:
    df1[column] = df1[column].astype('float32')

In [1291]:
train_total_seconds = df1['time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

In [1292]:
df1['total_sec'] = train_total_seconds

In [1293]:
# 월에 대한 One-hot encoding 구하기
months = pd.get_dummies(df1['time'].dt.month, prefix='month')

df1 = pd.concat([df1, months], axis=1)

In [1294]:
for i in range(1, 13):
    col_name = f"month_{i}"
    if col_name not in df1.columns:
        df1[col_name] = False 
        
    if df1[col_name].dtype == bool:
        df1[col_name] = df1[col_name].astype(np.float32)


### 안정성점수

In [1295]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler


SPS_norm = s_scaler.transform(df1[['SPS']])

IF_norm = i_scaler.transform(df1[['IF']])

spot = df1['SpotPrice'].to_numpy().reshape(-1, 1)

In [1296]:
# 안정성 점수 계산
df1['Stability_score'] = 5*(2*SPS_norm + IF_norm) - 2*spot

In [1297]:
df1['Stability_score_label'] = df1['Stability_score']

In [1298]:
df1[:5]

,InstanceType,time,SpotPrice,SPS,AZ,Region,IF,weekday,weekday_Friday,weekday_Monday,...,month_5,month_8,month_9,month_10,month_11,month_12,month_6,month_7,Stability_score,Stability_score_label
0,m4.4xlarge,2022-08-25 00:00:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
1,m4.4xlarge,2022-08-25 00:10:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
2,m4.4xlarge,2022-08-25 00:20:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
3,m4.4xlarge,2022-08-25 00:30:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
4,m4.4xlarge,2022-08-25 00:40:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516


In [1299]:
input_col = ['weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'total_sec', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'month_6', 'month_7', 'Stability_score']

target_col = ['Stability_score_label']

In [1300]:
from sklearn.preprocessing import StandardScaler
import pickle

path = "...scaler path..."

mm_scaler_x = MinMaxScaler()
mm_scaler_y = MinMaxScaler()
st_scaler = StandardScaler()
rb_scaler = RobustScaler()

with open(path+'...file name...', 'rb') as f:
    mm_scaler_x = pickle.load(f)
    
with open(path+'...file name...', 'rb') as f:
    mm_scaler_y = pickle.load(f)
    

In [1301]:
df1

,InstanceType,time,SpotPrice,SPS,AZ,Region,IF,weekday,weekday_Friday,weekday_Monday,...,month_5,month_8,month_9,month_10,month_11,month_12,month_6,month_7,Stability_score,Stability_score_label
0,m4.4xlarge,2022-08-25 00:00:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
1,m4.4xlarge,2022-08-25 00:10:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
2,m4.4xlarge,2022-08-25 00:20:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
3,m4.4xlarge,2022-08-25 00:30:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
4,m4.4xlarge,2022-08-25 00:40:00+00:00,0.242,3.0,apne2-az3,ap-northeast-2,3.0,Thursday,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,149.516,149.516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38955,m4.4xlarge,2023-05-22 05:40:00+00:00,0.346,2.0,apne2-az3,ap-northeast-2,1.0,Monday,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.308,59.308
38956,m4.4xlarge,2023-05-22 05:50:00+00:00,0.346,2.0,apne2-az3,ap-northeast-2,1.0,Monday,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.308,59.308
38957,m4.4xlarge,2023-05-22 06:00:00+00:00,0.346,2.0,apne2-az3,ap-northeast-2,1.0,Monday,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.308,59.308
38958,m4.4xlarge,2023-05-22 06:10:00+00:00,0.346,1.0,apne2-az3,ap-northeast-2,1.0,Monday,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.308,14.308


In [1330]:
split_date = '2023-05-20'

# train_df = df1[df1['time'] < split_date]
test_df = df1[df1['time'] >= split_date]

In [1331]:
len(test_df)

348

In [1332]:
test_df[:3]

,InstanceType,time,SpotPrice,SPS,AZ,Region,IF,weekday,weekday_Friday,weekday_Monday,...,month_5,month_8,month_9,month_10,month_11,month_12,month_6,month_7,Stability_score,Stability_score_label
38612,m4.4xlarge,2023-05-20 00:00:00+00:00,0.3509,1.0,apne2-az3,ap-northeast-2,1.0,Saturday,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.2982,14.2982
38613,m4.4xlarge,2023-05-20 00:10:00+00:00,0.3509,1.0,apne2-az3,ap-northeast-2,1.0,Saturday,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.2982,14.2982
38614,m4.4xlarge,2023-05-20 00:20:00+00:00,0.3509,2.0,apne2-az3,ap-northeast-2,1.0,Saturday,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.2982,59.2982


In [1333]:
test_X = test_df[input_col]
test_y = test_df[target_col]

In [1334]:
test_X[:144]

,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,total_sec,month_1,month_2,...,month_4,month_5,month_8,month_9,month_10,month_11,month_12,month_6,month_7,Stability_score
38612,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.2982
38613,0.0,0.0,1.0,0.0,0.0,0.0,0.0,600,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.2982
38614,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1200,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.2982
38615,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1200,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.2982
38616,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1800,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.2982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38751,0.0,0.0,1.0,0.0,0.0,0.0,0.0,76200,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.3052
38752,0.0,0.0,1.0,0.0,0.0,0.0,0.0,76800,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3052
38753,0.0,0.0,1.0,0.0,0.0,0.0,0.0,76800,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3052
38754,0.0,0.0,1.0,0.0,0.0,0.0,0.0,77400,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.3052


In [1335]:
input_size = 21
hidden_size = 100 
num_layers = 3 

output_size = 1 

In [1336]:
import torch
from model import *


path = "...model path..."

device = torch.device('cpu')
model = LSTM(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load(path + '...file name...', map_location=device))


<All keys matched successfully>

### 예측으로 예측하는 test 시각화 그래프

In [1337]:
from matplotlib import pyplot as plt

window_size = 6

# 예측값 그래프 함수
def plot_predictions(actual, predicted):
    plt.figure(figsize=(10,6)) #plotting

    plt.plot(actual, label='Actual Data') #actual plot 
    plt.plot(predicted, label='Predicted Data') #predicted plot
    plt.title('Time-Series Prediction')
    plt.legend()

    plt.ylim(bottom=min(min(actual), min(predicted))-1, top=max(max(actual), max(predicted))+1)

    plt.show() 
    

# 모델 예측값, 그래프 생성 함수
def visualize_predictions(model, test_X, test_y, window_size):
    model.eval()
    global pred
    global target
    
    predictions = []
    targets = []
    # 첫번째 시퀀스
    initial_sequence = np.array(test_X.iloc[:window_size])
    sequence = initial_sequence
    sequence = mm_scaler_x.transform(sequence)

    x = []
    for i in range(len(test_X) - window_size):

        input_data = torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)  # input 데이터

        prediction = model(input_data)
        predictions.append(prediction.item())
        
        # 다음 시퀀스 생성
        next_data = test_X.iloc[window_size + i, :-1]  # score를 제외한 다음 데이터
        
        prediction = mm_scaler_y.inverse_transform(prediction.detach().numpy())
        seq_next = mm_scaler_x.transform(np.array([np.concatenate((next_data, [prediction.item()]))]))
        sequence = np.concatenate((sequence[1:], seq_next))
        
        x.append(sequence)
        targets.append(np.array(test_y.iloc[i+window_size]))
    
    predictions = mm_scaler_y.inverse_transform(np.array(predictions).reshape(-1, 1))
    # 예측값 그래프 출력
    pred, target = predictions, targets
    plot_predictions(targets, predictions)

In [ ]:
visualize_predictions(model, test_X, test_y, window_size)

In [ ]:
mape = np.mean(np.abs((target - pred) / target)) * 100

print('MAPE:', mape)